In [2]:
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"]=(20,10)
import os
for dirname, _, filenames in os.walk('../input/bengaluru-house-price-data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
df1 =pd.read_csv("../input/bengaluru-house-price-data/Bengaluru_House_Data.csv")
df1.head()

In [3]:
df1.groupby('area_type')['area_type'].agg('count')

In [4]:
df2 = df1.drop(['area_type','society','balcony','availability'],axis='columns')
df2.head()

In [5]:
df2.isnull().sum()

In [6]:
df3=df2.dropna()
df3.isnull().sum()

In [7]:
df3['size'].unique()

In [8]:
df3['BHK'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))

In [9]:
df3.head()

In [10]:
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens)==2:
        return (float(tokens[0])+float(tokens[1]))/2
    try: 
        return float(x)
    except:
        return None

In [11]:
df4 = df3.copy()
df4['total_sqft'] = df4['total_sqft'].apply(convert_sqft_to_num)
df4.head()

In [12]:
df5 = df4.copy()
df5['price_per_sqft']=df5['price']*100000/df5['total_sqft']
df5.head()

In [13]:
df5.location = df5.location.apply(lambda x: x.strip())
location_stats =df5.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stats

In [14]:
location_stats_less_than_10 = location_stats[location_stats<=10]
df5.location= df5.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
df5.head()

In [15]:
df6=df5[~(df5.total_sqft/df5.BHK<300)]
df6.shape

In [16]:
df6.price_per_sqft.describe()

In [17]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
df7 = remove_pps_outliers(df6)
df7.shape

In [18]:
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location,location_df in df.groupby('location'):
        bhk_stats ={}
        for bhk,bhk_df in location_df.groupby('BHK'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk,bhk_df in location_df.groupby('BHK'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices,bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
df8 = remove_bhk_outliers(df7)
df8.shape

In [19]:
df8[df8.bath>df8.BHK+2]

In [20]:
df9=df8[df8.bath<df8.BHK+2]
df9.shape

In [21]:
df10 = df9.drop(['size','price_per_sqft'],axis='columns')
df10.head()

In [22]:
dummies = pd.get_dummies(df10.location)
dummies.head()

In [23]:
df11 = pd.concat([df10,dummies.drop('other',axis='columns')],axis='columns')
df11.head()

In [24]:
df12 = df11.drop('location',axis='columns')
df12.head()

In [25]:
x = df12.drop('price',axis='columns')
x.head()

In [26]:
y=df12.price
y.head()

In [27]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=10)

In [28]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(x_train,y_train)
lr_clf.score(x_test,y_test)

In [29]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
cv = ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
cross_val_score(LinearRegression(),x,y,cv=cv)

In [30]:
def predict_price(location,sqft,bath,BHK):
    loc_index = np.where(x.columns==location)[0][0]
    z=np.zeros(len(x.columns))
    z[0]=sqft
    z[1]=bath
    z[2]=BHK
    if loc_index >= 0:
        z[loc_index] = 1
    return lr_clf.predict([z])[0]
    

In [31]:
predict_price('1st Phase JP Nagar',1000,2,2)

In [32]:
import pickle
with open('bengaluru_home_prices_model.pickle','wb') as f:
    pickle.dump(lr_clf,f)

In [34]:
import json
columns = {
    'data_columns': [col.lower() for col in x.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))